In [9]:
import datetime
import pandas as pd
import numpy as np
from datascience import *
import random
from arcgis.gis import GIS

now = datetime.datetime.now()

# read in schools table
school_df = pd.read_pickle('geocoding_schools/cleaned_school_list.pk1')


#from arcgis.geocoding import geocoding
my_gis = GIS()
seattle = my_gis.map("Seattle, WA")

geocodes = school_df['geocode']
name = school_df['School']

i = 0
for school in geocodes:
    try:
        seattle.draw(school['location'], {"title" : name[i], "content" : school['address']},
                     {"type": "esriSMS","style": "esriSMSCircle","color": [0,0,0,125],"size": 4,
                      "angle": 0,"xoffset": 0,"yoffset": 0,
                      "outline": {"color": [0,0,0,255],"width": 0}})
    except:
        pass
        #print("No address for:", (name[i]))
    i+=1
    
late_bus_df = pd.read_pickle('scraping_bus_data/bus_data.pk1')
late_bus_df1  = late_bus_df.loc[late_bus_df['Day'] == 7]#str(now.day)]
late_bus_df2  = late_bus_df1.loc[late_bus_df1['To/From'] == 'to']
late_bus_df3  = late_bus_df2.loc[late_bus_df2['Data Taken Minute'] == 20] #20
late_schools = school_df.loc[school_df.get('School').isin(late_bus_df3.get('School'))]

# calculate the total bus late time for each school
total_rows = int(late_schools.size/8)

late_time = (total_rows)*[0]
i = 0

# add minute late column to school df
for index_schools, row_schools in late_schools.iterrows():
    for index_buses, row_buses in late_bus_df3.iterrows():
        if row_schools['School'] == row_buses['School']:
            #print(row_schools['School'],row_buses['School'])
            late_time[i] += row_buses['Time']
    i+=1
    #print(late_time)
    
late_schools['Late Time'] = late_time


# map late schools
geocodes = late_schools['geocode']
name = list(late_schools['School'])
late_time = list(late_schools['Late Time'])

#print(late_time)
i = 0
for school in geocodes:
    max_d = 8
    d = (max_d*(late_time[i]+30))/120
    seattle.draw(school['location'], {"title" : name[i], "content" : "Late: "+str(late_time[i])+" minutes"},
                     {"type": "esriSMS","style": "esriSMSCircle","color": [255,0,125,255],
                      "size": d,
                      "angle": 0,"xoffset": 0,"yoffset": 0,
                      "outline": {"color": [0,0,0,255],"width": 1}})

    i+=1
       
seattle



MapView(layout=Layout(height='400px', width='100%'))

In [10]:
late_bus_df

,Month,Day,Year,Bus Number,School,To/From,Time,Unit,Data Taken Month,Data Taken Day,Data Taken Year,Data Taken Hour,Data Taken Minute
0,December,21,2018,3,Adams,from,20,minutes,1,1,2019,20,51
1,December,21,2018,120,Loyal Heights,from,20,minutes,1,1,2019,20,51
2,December,21,2018,630,Queen Anne,from,20,minutes,1,1,2019,20,51
3,December,21,2018,148,Northgate,from,10,minutes,1,1,2019,20,51
4,December,21,2018,799,Gatzert,from,120,minutes,1,1,2019,20,51
5,December,21,2018,462,Washington,from,60,minutes,1,1,2019,20,51
6,December,21,2018,792,Kimball,from,120,minutes,1,1,2019,20,51
7,December,21,2018,402,Cleveland,from,60,minutes,1,1,2019,20,51
8,December,21,2018,624,Fairmount Park,from,120,minutes,1,1,2019,20,51
9,December,21,2018,464,Washington,from,60,minutes,1,1,2019,20,51


# commulative data

## late buses as of 7:20am each morning - starting January 7, 2018


In [ ]:
now = datetime.datetime.now()

# read in schools table
school_df = pd.read_pickle('geocoding_schools/cleaned_school_list.pk1')


#from arcgis.geocoding import geocoding
my_gis = GIS()
seattle2 = my_gis.map("Seattle, WA")

geocodes = school_df['geocode']
name = school_df['School']

i = 0
for school in geocodes:
    try:
        seattle2.draw(school['location'], {"title" : name[i], "content" : school['address']},
                     {"type": "esriSMS","style": "esriSMSCircle","color": [0,255,0,255],"size": 6,
                      "angle": 0,"xoffset": 0,"yoffset": 0,
                      "outline": {"color": [0,0,0,255],"width": 0}})
    except:
        pass
        #print("No address for:", (name[i]))
    i+=1
    
late_bus_df = pd.read_pickle('scraping_bus_data/bus_data.pk1')
#late_bus_df1  = late_bus_df.loc[late_bus_df['Day'] == str(now.day)]
late_bus_df2  = late_bus_df1.loc[late_bus_df1['To/From'] == 'to']
late_bus_df3  = late_bus_df2.loc[late_bus_df2['Data Taken Minute'] == 20] #20
late_schools = school_df.loc[school_df.get('School').isin(late_bus_df3.get('School'))]

# calculate the total bus late time for each school
total_rows = int(late_schools.size/8)

late_time = (total_rows)*[0]
i = 0

# add minute late column to school df
for index_schools, row_schools in late_schools.iterrows():
    for index_buses, row_buses in late_bus_df3.iterrows():
        if row_schools['School'] == row_buses['School']:
            #print(row_schools['School'],row_buses['School'])
            late_time[i] += row_buses['Time']
    i+=1
    #print(late_time)
    
late_schools['Late Time'] = late_time


# map late schools
geocodes = late_schools['geocode']
name = list(late_schools['School'])
late_time = list(late_schools['Late Time'])

#print(late_time)
i = 0
for school in geocodes:
    max_d = 8
    d = (max_d*(late_time[i]+30))/120
    seattle2.draw(school['location'], {"title" : name[i], "content" : "Late: "+str(late_time[i])+" minutes"},
                     {"type": "esriSMS","style": "esriSMSCircle","color": [255,0,125,255],
                      "size": d,
                      "angle": 0,"xoffset": 0,"yoffset": 0,
                      "outline": {"color": [0,0,0,255],"width": 1}})

    i+=1
       
seattle2

